In [32]:
import numpy as np
from scipy import optimize
from scipy.stats import multivariate_normal, norm
from scipy.integrate import dblquad, tplquad
import matplotlib.pyplot as plt

In [33]:
def compute_win_probabilities_analytical(prices, buyer):
    """
    Compute win probabilities analytically using multivariate Gaussian properties.
    
    Several approaches depending on your economic model:
    """
    prices = np.array(prices)
    
    # Method 1: Maximum of multivariate normal (most complex but exact)
    # P(max(V1, V2, V3) > p) for each price p
    def max_multivariate_cdf(price):
        # This requires numerical integration for general case
        # For 3D case, we integrate over the region where max(v1,v2,v3) <= price
        def integrand(v1, v2, v3):
            if max(v1, v2, v3) <= price:
                return multivariate_normal.pdf([v1, v2, v3], buyer.mu, buyer.sigma)
            return 0
        
        # Integration bounds (using 5 standard deviations as practical infinity)
        std_devs = np.sqrt(np.diag(buyer.sigma))
        lower_bounds = buyer.mu - 5 * std_devs
        upper_bounds = np.minimum(buyer.mu + 5 * std_devs, price)
        
        if np.any(upper_bounds <= lower_bounds):
            return 0.0
            
        try:
            result, _ = tplquad(integrand, 
                              lower_bounds[0], upper_bounds[0],
                              lower_bounds[1], upper_bounds[1], 
                              lower_bounds[2], upper_bounds[2])
            return 1 - result  # P(max > price) = 1 - P(max <= price)
        except:
            return 0.0
    
    # Method 2: Sum of multivariate normal (simpler, closed form)
    # If valuations represent different product attributes that sum up
    def sum_multivariate_normal(price):
        # V_total = V1 + V2 + V3 ~ N(sum(mu), ones^T * Sigma * ones)
        total_mean = np.sum(buyer.mu)
        ones = np.ones(len(buyer.mu))
        total_variance = ones.T @ buyer.sigma @ ones
        return 1 - norm.cdf(price, total_mean, np.sqrt(total_variance))
    
    # Method 3: Average of multivariate normal (closed form)
    # V_avg = (V1 + V2 + V3)/3 ~ N(mean(mu), (1/9) * ones^T * Sigma * ones)
    def avg_multivariate_normal(price):
        n = len(buyer.mu)
        avg_mean = np.mean(buyer.mu)
        ones = np.ones(n)
        avg_variance = (ones.T @ buyer.sigma @ ones) / (n**2)
        return 1 - norm.cdf(price, avg_mean, np.sqrt(avg_variance))
    
    # Method 4: Any dimension exceeds price (union of events)
    # P(V1 > p OR V2 > p OR V3 > p) = 1 - P(V1 <= p AND V2 <= p AND V3 <= p)
    def any_exceeds_price(price):
        # This is P(all <= price) using multivariate CDF
        upper_limits = np.full(len(buyer.mu), price)
        cdf_value = multivariate_normal.cdf(upper_limits, buyer.mu, buyer.sigma)
        return 1 - cdf_value
    
    print("Computing win probabilities using different methods:")
    
    # Choose your method based on economic interpretation:
    win_probs_max = []      # Method 1: Max valuation
    win_probs_sum = []      # Method 2: Sum valuation  
    win_probs_avg = []      # Method 3: Average valuation
    win_probs_any = []      # Method 4: Any exceeds
    
    for price in prices:
        # Method 1: Computationally expensive
        # win_probs_max.append(max_multivariate_cdf(price))
        
        # Method 2: Sum interpretation
        win_probs_sum.append(sum_multivariate_normal(price))
        
        # Method 3: Average interpretation  
        win_probs_avg.append(avg_multivariate_normal(price))
        
        # Method 4: Any dimension exceeds
        win_probs_any.append(any_exceeds_price(price))
    
    return {
        'sum': np.array(win_probs_sum),
        'average': np.array(win_probs_avg), 
        'any_exceeds': np.array(win_probs_any)
    }

In [34]:
def compare_methods(prices, rho, buyer, num_samples=10000):
    """Compare Monte Carlo vs Analytical methods"""
    
    # Monte Carlo (your original approach, fixed)
    valuations = np.random.multivariate_normal(buyer.mu, buyer.sigma, size=num_samples)
    max_valuations = np.max(valuations, axis=1)
    mc_win_probs = np.mean(max_valuations[:, np.newaxis] > np.array(prices), axis=0)
    
    # Analytical methods
    analytical_probs = compute_win_probabilities_analytical(prices, buyer)
    
    print("Method Comparison:")
    print(f"Monte Carlo (max):     {mc_win_probs}")
    print(f"Analytical (sum):      {analytical_probs['sum']}")
    print(f"Analytical (average):  {analytical_probs['average']}")  
    print(f"Analytical (any):      {analytical_probs['any_exceeds']}")
    
    return {
        'monte_carlo': mc_win_probs,
        'analytical': analytical_probs
    }

In [35]:
def compute_best_prices_analytical(prices, rho, buyer, method='average'):
    """
    Compute optimal pricing using analytical win probabilities.
    """
    win_prob_methods = compute_win_probabilities_analytical(prices, buyer)
    win_probabilities = win_prob_methods[method]
    
    print(f"Using method: {method}")
    print(f"Win probabilities: {win_probabilities}")
    
    prices = np.array(prices)
    
    # Linear programming setup (same as before)
    c = -(prices * win_probabilities)
    A_ub = [win_probabilities]
    b_ub = [rho]
    A_eq = [np.ones(len(prices))]
    b_eq = [1]
    bounds = [(0, 1)] * len(prices)
    
    res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub,
                          A_eq=A_eq, b_eq=b_eq,
                          bounds=bounds, method='highs')
    
    gamma = res.x
    expected_revenue = -res.fun
    expected_units_sold = np.sum(gamma * win_probabilities)
    
    return gamma, expected_revenue, expected_units_sold

In [2]:
class Buyer:
  def __init__(self, mu, sigma):
    """
    Buyer using a multivariate normal distribution for valuations.
    :param mu: Mean vector (list or 1D numpy array)
    :param sigma: Covariance matrix (2D numpy array)
    """
    self.mu = np.array(mu)
    self.sigma = np.array(sigma)
    self.dimension = self.mu.shape[0]

    # Validate that the covariance matrix is square and matches the mean vector
    if self.sigma.shape != (self.dimension, self.dimension):
      raise ValueError("Covariance matrix shape must match (len(mu), len(mu))")

  def round(self, prices):
    """
    Simulates a round where the buyer evaluates multiple prices.
    :param prices: List or array of offered prices (must match dimension)
    :return: Binary array indicating purchases (1 = buy, 0 = no buy)
    """
    valuations = np.random.multivariate_normal(self.mu, self.sigma)
    prices = np.array(prices)

    if len(prices) != self.dimension:
      raise ValueError("Length of prices must match length of mean vector")

    return (valuations > prices).astype(int)

In [ ]:
class CombinatorialCompanyUCB1:
  def __init__(self, K, B, T, N, range=1):
    self.K = K
    self.T = T
    self.range = range
    self.s_t = None # super_arm
    self.avg_f = np.zeros((N, K))
    self.avg_c = np.zeros((N, K))
    self.N_pulls = np.zeros((N, K))
    self.budget = B
    self.rho = B/T
    self.t = 0

  def pull_arm(self):
    if self.budget < 1:
      self.s_t = np.zeros(self.N)
      return self.s_t
    else:
      f_ucbs = self.avg_f + self.range*np.sqrt(2*np.log(self.T)/self.N_pulls)
      c_lcbs = self.avg_c - self.range*np.sqrt(2*np.log(self.T)/self.N_pulls)
      gamma_m = self.compute_opt(f_ucbs, c_lcbs)
      self.s_t = np.zeros(self.N)
      for i in range(self.N):
        gammp_t = gamma_m[i]
        self.s_t[i] = np.random.choice(self.K, p=gammp_t)
    return self.s_t

  def compute_combinatorial_opt(self, f_ucbs_matrix, c_lcbs_matrix):
        """
        f_ucbs_matrix: shape (N, K) - objective vectors for each product
        c_lcbs_matrix: shape (N, K) - constraint vectors for each product
        returns: gamma_matrix of shape (N, K) - optimal distributions
        """
        N, K = f_ucbs_matrix.shape
        gamma_matrix = np.zeros((N, K))

        for i in range(N):
            f_ucbs = f_ucbs_matrix[i]
            c_lcbs = c_lcbs_matrix[i]
            rho = self.rho_vec[i]

            # Special case: no cost constraint needed
            if np.all(c_lcbs <= 0):
                gamma = np.zeros(K)
                gamma[np.argmax(f_ucbs)] = 1.0
                gamma_matrix[i] = gamma
                continue

            # Set up the LP
            c = -f_ucbs
            A_ub = [c_lcbs]
            b_ub = [rho]
            A_eq = [np.ones(K)]
            b_eq = [1]
            bounds = [(0, 1) for _ in range(K)]

            res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub,
                                   A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

            if res.success:
                gamma_matrix[i] = res.x
            else:
                raise ValueError(f"LP failed for product {i}: {res.message}")

        return gamma_matrix

  def update(self, f_t, c_t):
    self.N_pulls[self.p_t] += 1
    self.avg_f[self.p_t] += (f_t - self.avg_f[self.p_t])/self.N_pulls[self.p_t]
    self.avg_c[self.p_t] += (c_t - self.avg_c[self.p_t])/self.N_pulls[self.p_t]
    self.budget -= c_t
    self.t += 1

In [ ]:
mu = [0.5, 0.2, 0.6]
sigma = np.array([[0.1, 0.05, 0.02],
                  [0.05, 0.1, 0.03],
                  [0.02, 0.03, 0.1]])
buyer = Buyer(mu, sigma)
prices = np.linspace(0, 1, 11)
rho = 0.4

# Compare methods
comparison = compare_methods(prices, rho, buyer)

# Solve using analytical method
gamma, revenue, units = compute_best_prices_analytical(prices, rho, buyer, method='average')
print(f"\nOptimal solution (analytical):")
print(f"Gamma: {gamma}")
print(f"Expected revenue: {revenue}")
print(f"Expected units sold: {units}")

n_trials = 10

regret_per_trial = []

for seed in range(n_trials):
  np.random.seed(seed)

  agent = CombinatorialCompanyUCB1(K=K, B=B, T=T, N=N)
  ag_utility = np.zeros(T)

  flag = True
  for i in range(T):
    arm = agent.pull_arm()
    price = P[arm]

    sold = buyer.round(price)

    f_t = price if sold else 0        # reward: revenue
    c_t = 1 if sold else 0            # cost: unit consumed

    agent.update(f_t=f_t, c_t=c_t)

    ag_utility[i] = f_t

    if agent.budget <= 0 and flag:
      print("TRIAL", seed , "Budget exhausted at round", i)
      flag = False

  cumulative_regret = np.cumsum(exp_utility-ag_utility)
  regret_per_trial.append(cumulative_regret)

regret_per_trial = np.array(regret_per_trial)

average_regret = regret_per_trial.mean(axis=0)
regret_std = regret_per_trial.std(axis=0)

print("Agent Cumulative Reward", np.sum(ag_utility))
print("Average cumulative regret", average_regret[-1])

Computing win probabilities using different methods:
Method Comparison:
Monte Carlo (max):     [0.9981 0.992  0.9763 0.9418 0.8817 0.7873 0.6667 0.5242 0.3785 0.2483
 0.1489]
Analytical (sum):      [0.96700397 0.95515699 0.94010253 0.9213504  0.89845411 0.87105048
 0.8389006  0.80192805 0.76024994 0.71419618 0.66431338]
Analytical (average):  [0.96700397 0.9213504  0.8389006  0.71419618 0.55623146 0.38864871
 0.23975006 0.12894952 0.05989747 0.02385744 0.00810477]
Analytical (any):      [0.99700489 0.99071382 0.97514545 0.94225792 0.88276728 0.79035216
 0.66657431 0.5230246  0.37821122 0.25044266 0.15129888]
Computing win probabilities using different methods:
Using method: average
Win probabilities: [0.96700397 0.9213504  0.8389006  0.71419618 0.55623146 0.38864871
 0.23975006 0.12894952 0.05989747 0.02385744 0.00810477]

Optimal solution (analytical):
Gamma: [0.         0.         0.         0.         0.06773546 0.93226454
 0.         0.         0.         0.         0.        ]
Exp